# Imports

**Importing the package you will need on the top of your notebook is a good programming practice** 

In [1]:
# Import the packages that will be usefull for this part of the lesson
from collections import OrderedDict, Counter
import pandas as pd
from pprint import pprint

# Small trick to get a larger display
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

 # General file parsing strategy

* **1. Read the first line of the file and try to understand the structure and determine the length of the file**

In [2]:
file = "../data/gencode_sample.gff3"

In [3]:
# Count all the lines in python
i = 0
with open (file, "r") as fp:
    for line in fp:
        i += 1
print (i, "lines found")

100 lines found


In [4]:
# Count all the lines in bash
!wc {file}

  100   900 48233 ../data/gencode_sample.gff3


In [5]:
# Read the first 2 lines in python
with open (file, "r") as fp:
    for _ in range (2):
        print (next(fp).strip())

chr1	HAVANA	exon	181715338	181715391	.	+	.	ID=exon:ENST00000367573.6:9;Parent=ENST00000367573.6;gene_id=ENSG00000198216.10;transcript_id=ENST00000367573.6;gene_type=protein_coding;gene_status=KNOWN;gene_name=CACNA1E;transcript_type=protein_coding;transcript_status=KNOWN;transcript_name=CACNA1E-003;exon_number=9;exon_id=ENSE00003728811.1;level=2;protein_id=ENSP00000356545.2;transcript_support_level=1;tag=non_canonical_U12,basic,appris_alternative_2,CCDS;ccdsid=CCDS55664.1;havana_gene=OTTHUMG00000037301.6;havana_transcript=OTTHUMT00000090793.2
chr1	HAVANA	transcript	26647447	26647681	.	+	.	ID=ENST00000443847.2;Parent=ENSG00000235069.2;gene_id=ENSG00000235069.2;transcript_id=ENST00000443847.2;gene_type=processed_pseudogene;gene_status=KNOWN;gene_name=RP4-785J1.1;transcript_type=processed_pseudogene;transcript_status=KNOWN;transcript_name=RP4-785J1.1-001;level=1;transcript_support_level=NA;ont=PGO:0000004;tag=pseudo_consens,basic;havana_gene=OTTHUMG00000057507.2;havana_transcript=OTTHUMT00

In [6]:
# Read the first 2 lines in Bash
!head {file} -n 2

chr1	HAVANA	exon	181715338	181715391	.	+	.	ID=exon:ENST00000367573.6:9;Parent=ENST00000367573.6;gene_id=ENSG00000198216.10;transcript_id=ENST00000367573.6;gene_type=protein_coding;gene_status=KNOWN;gene_name=CACNA1E;transcript_type=protein_coding;transcript_status=KNOWN;transcript_name=CACNA1E-003;exon_number=9;exon_id=ENSE00003728811.1;level=2;protein_id=ENSP00000356545.2;transcript_support_level=1;tag=non_canonical_U12,basic,appris_alternative_2,CCDS;ccdsid=CCDS55664.1;havana_gene=OTTHUMG00000037301.6;havana_transcript=OTTHUMT00000090793.2
chr1	HAVANA	transcript	26647447	26647681	.	+	.	ID=ENST00000443847.2;Parent=ENSG00000235069.2;gene_id=ENSG00000235069.2;transcript_id=ENST00000443847.2;gene_type=processed_pseudogene;gene_status=KNOWN;gene_name=RP4-785J1.1;transcript_type=processed_pseudogene;transcript_status=KNOWN;transcript_name=RP4-785J1.1-001;level=1;transcript_support_level=NA;ont=PGO:0000004;tag=pseudo_consens,basic;havana_gene=OTTHUMG00000057507.2;havana_transcript=OTTHUMT00

* **2. If the file is a standard genomic/proteomic format, read the documentation**

> There is no central database. You have to find it by yourself

> Wikipedia is usually a good starting point, but try to identify the orginal source which is usually more up to date.

* **3. Think about the most efficient way to parse the file to get the information you want**

> How are you going to access the field(s) of interest ? (you can test that with 1 line before starting with the whole file)
    
> A real life file will contain millions of lines and file reading is usually slow. Try to read the file only 1 time, even if you need to parse multiple element per line. 
    
> How are you going to collect the information (dictionary, list, dataframe...) ?
    
**... Now you can parse the file**

# High-performance container datatypes from the standard library

**Python standard library contains a very interesting module to simplify the parsing a file: "collections" (See [documentation](https://docs.python.org/3.5/library/collections.html) for detailed information)**

**This module implements specialized container datatypes providing alternatives to Python’s native data structures (dict, list, set...)**

| **Collection** 	| **Functionality**                                                    	|
|------------------ |----------------------------------------------------------------------	|
| **Counter**     	| Dict subclass for counting hashable objects                          	|
| **OrderedDict** 	| Dict subclass that remembers the order entries were added            	|
| **namedtuple**  	| Factory function for creating tuple subclasses with named fields     	|
| **deque**       	| List-like container with fast appends and pops on either end         	|
| **ChainMap**    	| Dict-like class for creating a single view of multiple mappings      	|
| **defaultdict** 	| Dict subclass that calls a factory function to supply missing values 	|
| **UserDict**    	| Wrapper around dictionary objects for easier dict subclassing        	|
| **UserList**    	| Wrapper around list objects for easier list subclassing              	|
| **UserString**  	| Wrapper around string objects for easier string subclassing          	|

---
## Counter

**A [Counter](https://docs.python.org/3.5/library/collections.html#collections.Counter) container is provided to support convenient and rapid counting of specific occurences. See also [defaultdict](https://docs.python.org/3.5/library/collections.html#collections.defaultdict) for a generalization to other types than integer.**

***Example: counting characters in a string*** 

In [7]:
random_text = """Ukip is likely to be asked to repay tens of thousands of euros by European parliament finance chiefs
who have accused the party of misspending EU funds on party workers and Nigel Farage’s failed bid to win a seat in
Westminster.The Alliance for Direct Democracy in Europe (ADDE), a Ukip-dominated political vehicle, will be asked to
repay €173,000 (£148,000) in misspent funds and denied a further €501,000 in EU grants for breaking European rules
that ban spending EU money on national election campaigns and referendums. According to a European parliament audit
report seen by the Guardian, Ukip spent EU funds on polling and analysis in constituencies where they hoped to win a 
seat in the 2015 general election, including the South Thanet seat that party leader Farage contested. The party also
funded polls to gauge the public mood on leaving the EU, months before the official campaign kicked off in April 2016"""

In [8]:
# Example with a Counter 
c = Counter()

# Iterate over each characters of the string
for character in random_text:
    # Increment the counter for the current element
    c[character.lower()] += 1

# Order by most frequent element
c.most_common()

[(' ', 145),
 ('e', 93),
 ('a', 67),
 ('n', 66),
 ('i', 55),
 ('t', 54),
 ('o', 46),
 ('s', 39),
 ('r', 38),
 ('d', 35),
 ('l', 30),
 ('u', 29),
 ('p', 28),
 ('c', 23),
 ('h', 23),
 ('f', 21),
 ('g', 17),
 ('y', 13),
 ('m', 13),
 ('0', 12),
 ('k', 10),
 ('b', 9),
 (',', 8),
 ('w', 7),
 ('\n', 7),
 ('1', 5),
 ('.', 3),
 ('v', 3),
 ('5', 2),
 (')', 2),
 ('€', 2),
 ('(', 2),
 ('2', 2),
 ('£', 1),
 ('8', 1),
 ('4', 1),
 ('’', 1),
 ('-', 1),
 ('6', 1),
 ('7', 1),
 ('3', 1)]

In [9]:
# Same thing but with native collections
d = {}

# Iterate over each characters of the string
for character in random_text:
    # If the element is not in the dict we have to create an entry first
    if character not in d:
        d[character.lower()] = 0
    # Increment the counter for the current element
    d[character.lower()]+=1
    
# Order by most frequent element
sorted(d.items(), key=lambda t: t[1], reverse=True)

[(' ', 145),
 ('i', 55),
 ('n', 55),
 ('o', 46),
 ('r', 38),
 ('l', 30),
 ('p', 28),
 ('d', 24),
 ('c', 23),
 ('h', 23),
 ('y', 13),
 ('m', 13),
 ('0', 12),
 ('k', 10),
 ('b', 9),
 ('g', 9),
 (',', 8),
 ('t', 7),
 ('\n', 7),
 ('f', 7),
 ('s', 6),
 ('1', 5),
 ('e', 5),
 ('w', 4),
 ('.', 3),
 ('v', 3),
 ('5', 2),
 (')', 2),
 ('€', 2),
 ('(', 2),
 ('2', 2),
 ('£', 1),
 ('u', 1),
 ('8', 1),
 ('4', 1),
 ('’', 1),
 ('a', 1),
 ('-', 1),
 ('6', 1),
 ('7', 1),
 ('3', 1)]

---
## OrderedDict

**In a standard python dictionary, the order of the elements is not guaranteed and can change between 2 successive calls. In many situations, it can be annoying particularly if the order of elements in a parsed file matters (fastq, fasta...)**

**[Ordered dictionaries](https://docs.python.org/3.5/library/collections.html#collections.OrderedDict) are just like regular dictionaries but they remember the order that items were inserted, like lists.**

**When iterating over an ordered dictionary, the items are returned in the order their keys were first added.**

In [10]:
fruit_str = "banana ripe:banana unripe:banana ripe:banana rotten:apple ripe:apple ripe:apple ripe:apple unripe:orange unripe:orange unripe:orange unripe:pear rotten:pear rotten:pear ripe"

***Parsing with a normal dictionary*** 

In [11]:
d={}

for element in fruit_str.split(":"):
    fruit, status = element.split(" ")
    if fruit not in d:
        d[fruit] = Counter()
    d[fruit][status]+=1

d

{'apple': Counter({'ripe': 3, 'unripe': 1}),
 'banana': Counter({'ripe': 2, 'rotten': 1, 'unripe': 1}),
 'orange': Counter({'unripe': 3}),
 'pear': Counter({'ripe': 1, 'rotten': 2})}

***Parsing with a Ordered dictionary*** 

In [12]:
d=OrderedDict()

for element in fruit_str.split(":"):
    fruit, status = element.split(" ")
    if fruit not in d:
        d[fruit] = Counter()
    d[fruit][status]+=1

d

OrderedDict([('banana', Counter({'ripe': 2, 'rotten': 1, 'unripe': 1})),
             ('apple', Counter({'ripe': 3, 'unripe': 1})),
             ('orange', Counter({'unripe': 3})),
             ('pear', Counter({'ripe': 1, 'rotten': 2}))])

**Since an ordered dictionary remembers its insertion order, it can be used in conjunction with sorting to make a sorted dictionary (by key or value) from a standard dictionary**

In [13]:
print ("\nStandard unsorted dictionary")
d = {'banana':3, 'apple':4, 'pear':1, 'orange':2, "peach":10, "apricot":2}
pprint (d)

print("\nDictionary sorted by key")
d_per_key = OrderedDict(sorted(d.items(), key=lambda t: t[0]))
pprint (d_per_key)

print("\nDictionary sorted by value")
d_per_val = OrderedDict(sorted(d.items(), key=lambda t: t[1]))
pprint (d_per_val)


Standard unsorted dictionary
{'apple': 4, 'apricot': 2, 'banana': 3, 'orange': 2, 'peach': 10, 'pear': 1}

Dictionary sorted by key
OrderedDict([('apple', 4),
             ('apricot', 2),
             ('banana', 3),
             ('orange', 2),
             ('peach', 10),
             ('pear', 1)])

Dictionary sorted by value
OrderedDict([('pear', 1),
             ('apricot', 2),
             ('orange', 2),
             ('banana', 3),
             ('apple', 4),
             ('peach', 10)])


---
# Pandas: Powerful data structures for data analysis, time series, and statistics

* Flexible, and expressive data structures **Series** (1D) and **DataFrame** (2D) and **Panel** (3D) 
* High-level building block for doing **practical, real world data analysis**
* **Nearly as fast as C language** = Build on top of Numpy and extensive use of Cython
* **Robust IO tools** for loading and parsing data from text files, excel files and databases.

## Introduction to Series

* **1D labeled array capable of holding any data type** (integers, strings, float...)

* Similar to a python standard dictionary but **faster** (because based on C datatypes) and more **user-friendly** in Jupyter

### Create series

***From 2 lists or sets ***

In [14]:
Base = ('A','T','C','G','N')
Freq = (0.21, 0.24, 0.27, 0.25, 0.03)
pd.Series(data=Freq, index=Base)

A    0.21
T    0.24
C    0.27
G    0.25
N    0.03
dtype: float64

***From a python dictionary***

In [15]:
d = {'A':0.21, 'T':0.24, 'C':0.27, 'G':0.25, 'N':0.03}
pd.Series(d)

A    0.21
C    0.27
G    0.25
N    0.03
T    0.24
dtype: float64

***The data type and series names can be specified*** 

In [16]:
d = {'A':21.0, 'T':24.0, 'C':27.0, 'G':25.0, 'N':3.0}
pd.Series(d, name="Percent", dtype=int)

A    21
C    27
G    25
N     3
T    24
Name: Percent, dtype: int64

**From a file containing 2 columns with the squeeze option**

In [17]:
pd.read_table("../data/DNA_distrib.tsv", index_col=0, squeeze=True, sep="\t")

base
A    0.21
T    0.24
C    0.27
G    0.25
N    0.03
Name: freq, dtype: float64

### Manipulate series

***Support list methods***

In [18]:
s = pd.Series({'A':0.21, 'T':0.24, 'C':0.27, 'G':0.25})

# Concat 2 series
s2 = pd.Series({'Y':0.01, 'N':0.03})
s3 = s.append(s2)
print(s3)

# Slicing
print(s[2:4])

# Extraction
print(s[2])

# the "for" loop works as for a list 
for i in s:
    print (i)


A    0.21
C    0.27
G    0.25
T    0.24
N    0.03
Y    0.01
dtype: float64
G    0.25
T    0.24
dtype: float64
0.25
0.21
0.27
0.25
0.24


***Support dictionary methods***

In [19]:
s = pd.Series({'A':21.0, 'T':24.0, 'C':27.0, 'G':25.0, 'N':3.0}, name="Percent", dtype=int)

# Update value
s["A"] = 22
print(s)

# Named indexing
print(s["A"])

# Test for existence
print ("A" in s)
print ("V" in s)

A    22
C    27
G    25
N     3
T    24
Name: Percent, dtype: int64
22
True
False


***Support a wide range of mathematic operations (thanks to numpy)***

In [20]:
s = pd.Series({'A':21, 'T':24, 'C':27, 'G':25, 'N':3}, name="Percent")

print(s.max())
print(s.mean())
print(s.all()>20)
print(s.sem())

# Addition of 2 series will return a results for all values in the 2 series
s2 = pd.Series({'A':0.2, 'T':0.7, 'C':0.4, 'G':1.5, 'N':-3}, name="Percent")

print (s + s2)

27
20.0
False
4.35889894354
A    21.2
C    27.4
G    26.5
N     0.0
T    24.7
Name: Percent, dtype: float64


## Introduction to Dataframes

* **2-dimensional labeled data structure with columns of potentially different types**
* **HTML rendering in jupyter**

### Create dataframe from another datatypes

***You can optionally pass index (row labels) and columns (column labels) arguments***

***From a pandas Series***

In [21]:
s = pd.Series({'A':21.0, 'T':24.0, 'C':27.0, 'G':25.0, 'N':3.0})
pd.DataFrame(s, columns=["Percent"])

,Percent
A,21
C,27
G,25
N,3
T,24


***From a list of pandas Series***

In [22]:
series_list = [
    pd.Series({'A':21.0, 'T':24.0, 'C':27.0, 'G':25.0, 'N':3.0}, name="Percent"),
    pd.Series({'A':331.2, 'T':322.2, 'C':307.2, 'G':347.2, 'N':None}, name="MolecularWeight"),
    pd.Series({'A':259, 'T':267, 'C':271, 'G':253, 'N':None}, name="AbsorbanceMax")]

pd.DataFrame(series_list)

,A,C,G,N,T
Percent,21.0,27.0,25.0,3,24.0
MolecularWeight,331.2,307.2,347.2,NaN,322.2
AbsorbanceMax,259.0,271.0,253.0,NaN,267.0


**From a simple list of list** 

In [23]:
list_list = [[21.0, 24.0, 27.0, 25.0, 3.0], [331.2, 322.2, 307.2, 347.2, None], [259, 267, 271, 253, None]]
column_list = ['A', 'T', 'C', 'G', 'N']
index_list = ["Percent", "MolecularWeight", "AbsorbanceMax"]

pd.DataFrame(list_list, index=index_list, columns=column_list)

,A,T,C,G,N
Percent,21.0,24.0,27.0,25.0,3
MolecularWeight,331.2,322.2,307.2,347.2,NaN
AbsorbanceMax,259.0,267.0,271.0,253.0,NaN


***The Dataframe creation is very versatile and can also be done from Dictionaries of lists, dicts, or Series and from numpy.ndarray...***

***The orientation may differ depending on the source data (columns or index orientation)***

In [24]:
dict_list = { "Percent":[21.0, 24.0, 27.0, 25.0, 3.0], "MolecularWeight":[331.2, 322.2, 307.2, 347.2, None], "AbsorbanceMax":[259, 267, 271, 253, None]}

pd.DataFrame(dict_list, )

,AbsorbanceMax,MolecularWeight,Percent
0,259,331.2,21
1,267,322.2,24
2,271,307.2,27
3,253,347.2,25
4,NaN,NaN,3


***One of the major strength of Pandas is its ability to perform complex files parsing into a comprehensive dataframe format***

***Example with a gff3 file***

In [25]:
file = "../data/gencode_sample.gff3"

*The file is a standard genomic/proteomic format*

>[gff3 documentation](http://gmod.org/wiki/GFF3)

>In this case the gff3 format consists of 9 tab delimited fields

>The standard fields are named "seqid", "source", "type", "start", "end", "score", "strand", "phase", "attributes"

*The hard way with a list of dictionaries... But you control everything and you can parse very complex structured files*

In [26]:
#Empty list to collect each lines 
l=[]

with open (file, "r") as fp:
    for line in fp:
        
        # Split the line by tabulations 
        sl = line.split("\t")
        
        # Using an ordered dictionary because we want to retain the order in the file
        line_dict = OrderedDict({
            "seqid":sl[0],  
            "source":sl[1],  
            "type":sl[2],  
            "start":sl[3],  
            "end":sl[4],  
            "score":sl[5],  
            "strand":sl[6],  
            "phase":sl[7],  
            "attributes":sl[8]})
        
        # Add the line to the general list
        l.append(line_dict)

# print only the 5 first lines of the list (because 10000 lines of dictionaries is a bit long)
pprint (l[0:5])

[OrderedDict([('attributes',
               'ID=exon:ENST00000367573.6:9;Parent=ENST00000367573.6;gene_id=ENSG00000198216.10;transcript_id=ENST00000367573.6;gene_type=protein_coding;gene_status=KNOWN;gene_name=CACNA1E;transcript_type=protein_coding;transcript_status=KNOWN;transcript_name=CACNA1E-003;exon_number=9;exon_id=ENSE00003728811.1;level=2;protein_id=ENSP00000356545.2;transcript_support_level=1;tag=non_canonical_U12,basic,appris_alternative_2,CCDS;ccdsid=CCDS55664.1;havana_gene=OTTHUMG00000037301.6;havana_transcript=OTTHUMT00000090793.2\n'),
              ('phase', '.'),
              ('seqid', 'chr1'),
              ('strand', '+'),
              ('end', '181715391'),
              ('score', '.'),
              ('start', '181715338'),
              ('source', 'HAVANA'),
              ('type', 'exon')]),
 OrderedDict([('attributes',
               'ID=ENST00000443847.2;Parent=ENSG00000235069.2;gene_id=ENSG00000235069.2;transcript_id=ENST00000443847.2;gene_type=processed_pseudoge

In [27]:
# Acces element
l[0]["seqid"]

'chr1'

In [28]:
# Cast in a dataframe
df = pd.DataFrame(l)
# print only the 10 first rows of the table (yes 10000 lines is still a bit long even in a dataframe)
df.head(10)

,attributes,end,phase,score,seqid,source,start,strand,type
0,ID=exon:ENST00000367573.6:9;Parent=ENST0000036...,181715391,.,.,chr1,HAVANA,181715338,+,exon
1,ID=ENST00000443847.2;Parent=ENSG00000235069.2;...,26647681,.,.,chr1,HAVANA,26647447,+,transcript
2,ID=UTR5:ENST00000370684.5;Parent=ENST000003706...,84164412,.,.,chr1,HAVANA,84164317,+,five_prime_UTR
3,ID=exon:ENST00000464631.6:4;Parent=ENST0000046...,177965167,.,.,chr1,HAVANA,177965047,-,exon
4,ID=CDS:ENST00000358779.9;Parent=ENST0000035877...,1708245,2,.,chr1,HAVANA,1708180,-,CDS
5,ID=ENST00000622395.4;Parent=ENSG00000158850.14...,161177968,.,.,chr1,ENSEMBL,161171311,-,transcript
6,ID=stop_codon:ENST00000622561.4;Parent=ENST000...,25317052,0,.,chr1,ENSEMBL,25317050,+,stop_codon
7,ID=ENST00000413093.2;Parent=ENSG00000142973.12...,46790499,.,.,chr1,HAVANA,46757878,+,transcript
8,ID=exon:ENST00000395445.5:7;Parent=ENST0000039...,54329706,.,.,chr10,HAVANA,54329596,-,exon
9,ID=exon:ENST00000434147.1:2;Parent=ENST0000043...,73814737,.,.,chr10,HAVANA,73814429,+,exon


*The easy way with a pandas Dataframe... Works most of the time for standard formats*

In [29]:
# Parse all the line with pandas
df = pd.read_table(file, sep="\t", names =["seqid", "source", "type", "start", "end", "score", "strand", "phase", "attributes"])

# print only the 10 first rows of the table (well, you know why now...)
df.head(10)

,seqid,source,type,start,end,score,strand,phase,attributes
0,chr1,HAVANA,exon,181715338,181715391,.,+,.,ID=exon:ENST00000367573.6:9;Parent=ENST0000036...
1,chr1,HAVANA,transcript,26647447,26647681,.,+,.,ID=ENST00000443847.2;Parent=ENSG00000235069.2;...
2,chr1,HAVANA,five_prime_UTR,84164317,84164412,.,+,.,ID=UTR5:ENST00000370684.5;Parent=ENST000003706...
3,chr1,HAVANA,exon,177965047,177965167,.,-,.,ID=exon:ENST00000464631.6:4;Parent=ENST0000046...
4,chr1,HAVANA,CDS,1708180,1708245,.,-,2,ID=CDS:ENST00000358779.9;Parent=ENST0000035877...
5,chr1,ENSEMBL,transcript,161171311,161177968,.,-,.,ID=ENST00000622395.4;Parent=ENSG00000158850.14...
6,chr1,ENSEMBL,stop_codon,25317050,25317052,.,+,0,ID=stop_codon:ENST00000622561.4;Parent=ENST000...
7,chr1,HAVANA,transcript,46757878,46790499,.,+,.,ID=ENST00000413093.2;Parent=ENSG00000142973.12...
8,chr10,HAVANA,exon,54329596,54329706,.,-,.,ID=exon:ENST00000395445.5:7;Parent=ENST0000039...
9,chr10,HAVANA,exon,73814429,73814737,.,+,.,ID=exon:ENST00000434147.1:2;Parent=ENST0000043...


### Viewing Data in a Dataframe

***See the top & bottom rows of the frame***

In [30]:
df.head(2)

,seqid,source,type,start,end,score,strand,phase,attributes
0,chr1,HAVANA,exon,181715338,181715391,.,+,.,ID=exon:ENST00000367573.6:9;Parent=ENST0000036...
1,chr1,HAVANA,transcript,26647447,26647681,.,+,.,ID=ENST00000443847.2;Parent=ENSG00000235069.2;...


In [31]:
df.tail(2)

,seqid,source,type,start,end,score,strand,phase,attributes
98,chrX,HAVANA,CDS,1208806,1208905,.,-,2,ID=CDS:ENST00000467626.6;Parent=ENST0000046762...
99,chrY,HAVANA,exon,2719661,2719705,.,+,.,ID=exon:ENST00000381192.8_PAR_Y:4;Parent=ENST0...


***Transposing your data***

In [32]:
df.transpose().head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
seqid,chr1,chr1,chr1,chr1,chr1,chr1,chr1,chr1,chr10,chr10,...,chr7,chr7,chr7,chr8,chr8,chr8,chrX,chrX,chrX,chrY
source,HAVANA,HAVANA,HAVANA,HAVANA,HAVANA,ENSEMBL,ENSEMBL,HAVANA,HAVANA,HAVANA,...,HAVANA,HAVANA,HAVANA,HAVANA,HAVANA,HAVANA,HAVANA,HAVANA,HAVANA,HAVANA
type,exon,transcript,five_prime_UTR,exon,CDS,transcript,stop_codon,transcript,exon,exon,...,gene,CDS,exon,five_prime_UTR,CDS,exon,start_codon,exon,CDS,exon
start,181715338,26647447,84164317,177965047,1708180,161171311,25317050,46757878,54329596,73814429,...,101162509,84129123,140478600,116938199,42350007,11853329,129523368,151923600,1208806,2719661
end,181715391,26647681,84164412,177965167,1708245,161177968,25317052,46790499,54329706,73814737,...,101165593,84129185,140479066,116938243,42350065,11853479,129523370,151925170,1208905,2719705


***Sorting data***

In [33]:
df.sort_values(by="seqid").head()

,seqid,source,type,start,end,score,strand,phase,attributes
0,chr1,HAVANA,exon,181715338,181715391,.,+,.,ID=exon:ENST00000367573.6:9;Parent=ENST0000036...
1,chr1,HAVANA,transcript,26647447,26647681,.,+,.,ID=ENST00000443847.2;Parent=ENSG00000235069.2;...
2,chr1,HAVANA,five_prime_UTR,84164317,84164412,.,+,.,ID=UTR5:ENST00000370684.5;Parent=ENST000003706...
3,chr1,HAVANA,exon,177965047,177965167,.,-,.,ID=exon:ENST00000464631.6:4;Parent=ENST0000046...
4,chr1,HAVANA,CDS,1708180,1708245,.,-,2,ID=CDS:ENST00000358779.9;Parent=ENST0000035877...


***sampling random rows or column: Awesomeness level +++***

In [34]:
df.sample(n=10, axis=0).sample(n=2, axis=1)

,seqid,score
54,chr2,.
64,chr20,.
94,chr8,.
26,chr14,.
89,chr7,.
47,chr19,.
13,chr11,.
44,chr18,.
59,chr2,.
75,chr5,.


### Descriptive statistics of a dataframe

**Similar to Series, many basic statistics are available.**

The results will only contain the relevant columns (for example *mean()* will be applied only to numeric columns.
 
| Function 	| Description                                	|
|----------	|--------------------------------------------	|
| count    	| Number of non-null observations            	|
| sum      	| Sum of values                              	|
| mean     	| Mean of values                             	|
| mad      	| Mean absolute deviation                    	|
| median   	| Arithmetic median of values                	|
| min      	| Minimum                                    	|
| max      	| Maximum                                    	|
| mode     	| Mode                                       	|
| abs      	| Absolute Value                             	|
| prod     	| Product of values                          	|
| std      	| Bessel-corrected sample standard deviation 	|
| var      	| Unbiased variance                          	|
| sem      	| Standard error of the mean                 	|
| skew     	| Sample skewness (3rd moment)               	|
| kurt     	| Sample kurtosis (4th moment)               	|
| unique    | List the unique elements 
| quantile 	| Sample quantile (value at %)               	|
| cumsum   	| Cumulative sum                             	|
| cumprod  	| Cumulative product                         	|
| cummax   	| Cumulative maximum                         	|
| cummin   	| Cumulative minimum                         	|

In [35]:
df.mean()

start    71288610.81
end      71292505.97
dtype: float64

In [36]:
df.count()

seqid         100
source        100
type          100
start         100
end           100
score         100
strand        100
phase         100
attributes    100
dtype: int64

**One can also use the *describe()* method to a simple table report of the dataframe**

In [37]:
df.describe(include="all")

,seqid,source,type,start,end,score,strand,phase,attributes
count,100,100,100,1.000000e+02,1.000000e+02,100,100,100,100
unique,22,2,8,NaN,NaN,1,2,4,100
top,chr7,HAVANA,exon,NaN,NaN,.,-,.,ID=CDS:ENST00000522610.5;Parent=ENST0000052261...
freq,10,93,55,NaN,NaN,100,54,78,1
mean,NaN,NaN,NaN,7.128861e+07,7.129251e+07,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,5.633092e+07,5.632923e+07,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,8.917500e+05,8.917520e+05,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,3.208370e+07,3.208377e+07,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,5.730800e+07,5.730990e+07,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,1.046151e+08,1.046156e+08,NaN,NaN,NaN,NaN


### Access the  element in a Dataframe = Indexing and Selection

| Operation                      	| Syntax        	| Result    	|
|--------------------------------	|---------------	|-----------	|
| Select column                  	| df[col]       	| Series    	|
| Select row by label            	| df.loc[label] 	| Series    	|
| Select row by integer location 	| df.iloc[loc]  	| Series    	|
| Slice rows                     	| df[5:10]      	| DataFrame 	|

***Loading a new example file formated in SAM format***

In [38]:
file =  "../data/sample_alignment.sam"

In [39]:
!wc {file}

  18  190 4858 ../data/sample_alignment.sam


In [40]:
!head -n 5 "../data/sample_alignment.sam"

@SQ	SN:SSV9K2-CMV-GFP-HygroTK-bGHpA	LN:7003
@PG	ID:bwa	PN:bwa	VN:0.7.10-r789	CL:bwa-0.7.10 mem -t 12 ../bwa_index/SSV9K2-CMV-GFP-HygroTK-bGHpA.fa ../fastq/RUN2_S1_R1_1_filtered.fastq.gz ../fastq/RUN2_S1_R2_2_filtered.fastq.gz
HWI-1KL149:87:HA58EADXX:1:1101:1531:2163	163	SSV9K2-CMV-GFP-HygroTK-bGHpA	4862	60	101M	=	4950	189	CGCCCTCCTGTGCTACCCGGCCGCGCGGTACCTTATGGGCAGCATGACCCCCCAGGCCGTGCTGGCGTTCGTGGCCCTCATCCCGCCGACCTTGCCCGGCA	CCCFFFFFGHHHHJJJJJJJJJJJJJJJGIJJJJJJHHHHHFFFEEEEEEDDDDBDDDDBBDDDDDDDDDDBDDDDDDDBDDDDDDDDDDDDDDDDDDDDB
HWI-1KL149:87:HA58EADXX:1:1101:1744:2169	141	*	0	0	*	*	0	0	TCGTAGATTTCTCTGGCGATTGAAGGGCTAAATTCTTCAACGCTAACTTTGAGAATTTTTGTAAGCAATGCGGCGTTATAAGCATTTAATGCATTGATGCC	CCCFFFFFHHHHHJJJJJJJJJJJJJJJJJJJIJJJJJJJJJJJJJJJIJJJJJJJJJJJIHHHHHHHFFFFDDDDDDDEEDDDDDDEEDDDDEEDDEDDD
HWI-1KL149:87:HA58EADXX:1:1101:1606:2189	83	SSV9K2-CMV-GFP-HygroTK-bGHpA	4285	60	101M	=	4104	-282	AGGGTCGATGCGACGCAATCGTCCGATCCGGAGCCGGGACTGTCGGGCGTACACAAATCGCCCGCAGAAGCGCGGCCGTCTGGACCGATGGCTGTGTAGAA	DDDDDDDDD

[SAM format documentation](http://genome.sph.umich.edu/wiki/SAM)

In [41]:
file = "../data/sample_alignment.sam"
columns_names = ['QNAME', 'FLAG', 'RNAME', 'POS', 'MAPQ', 'CIGAR', 'RNEXT', 'PNEXT', 'TLEN', 'SEQ', 'QUAL']
df = pd.read_table(file, sep="\t", names = columns_names, skiprows=[0,1], index_col=0)
df.head()

,FLAG,RNAME,POS,MAPQ,CIGAR,RNEXT,PNEXT,TLEN,SEQ,QUAL
QNAME,,,,,,,,,,
HWI-1KL149:87:HA58EADXX:1:1101:1531:2163,163,SSV9K2-CMV-GFP-HygroTK-bGHpA,4862,60,101M,=,4950,189,CGCCCTCCTGTGCTACCCGGCCGCGCGGTACCTTATGGGCAGCATG...,CCCFFFFFGHHHHJJJJJJJJJJJJJJJGIJJJJJJHHHHHFFFEE...
HWI-1KL149:87:HA58EADXX:1:1101:1744:2169,141,*,0,0,*,*,0,0,TCGTAGATTTCTCTGGCGATTGAAGGGCTAAATTCTTCAACGCTAA...,CCCFFFFFHHHHHJJJJJJJJJJJJJJJJJJJIJJJJJJJJJJJJJ...
HWI-1KL149:87:HA58EADXX:1:1101:1606:2189,83,SSV9K2-CMV-GFP-HygroTK-bGHpA,4285,60,101M,=,4104,-282,AGGGTCGATGCGACGCAATCGTCCGATCCGGAGCCGGGACTGTCGG...,DDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDD...
HWI-1KL149:87:HA58EADXX:1:1101:1752:2238,83,SSV9K2-CMV-GFP-HygroTK-bGHpA,3032,60,101M,=,2809,-324,CTGTTGAATGTCGTGAAGGAAGCAGTTCCTCTGGAAGCTTCTTGAA...,DDDEEDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDEDDD...
HWI-1KL149:87:HA58EADXX:1:1101:1752:2238,163,SSV9K2-CMV-GFP-HygroTK-bGHpA,2809,60,101M,=,3032,324,CTATAGGGAGACCACAACGGTTTCCCTCTAGCGGGATCAATTCCGC...,CCCFFFFFHHHHHJJIJJJJIIJJJJJJJJJJJJJHJJIJJJIIIJ...


***Examples of column slicing***

In [42]:
df["FLAG"]

QNAME
HWI-1KL149:87:HA58EADXX:1:1101:1531:2163     163
HWI-1KL149:87:HA58EADXX:1:1101:1744:2169     141
HWI-1KL149:87:HA58EADXX:1:1101:1606:2189      83
HWI-1KL149:87:HA58EADXX:1:1101:1752:2238      83
HWI-1KL149:87:HA58EADXX:1:1101:1752:2238     163
HWI-1KL149:87:HA58EADXX:1:1101:1813:2228    2115
HWI-1KL149:87:HA58EADXX:1:1101:2031:2119      99
HWI-1KL149:87:HA58EADXX:1:1101:2133:2244     163
HWI-1KL149:87:HA58EADXX:1:1101:2228:2222     141
HWI-1KL149:87:HA58EADXX:1:1101:2344:2177      83
HWI-1KL149:87:HA58EADXX:1:1101:2645:2113     163
HWI-1KL149:87:HA58EADXX:1:1101:2485:2160     163
HWI-1KL149:87:HA58EADXX:1:1101:2647:2140      83
HWI-1KL149:87:HA58EADXX:1:1101:2813:2174      99
HWI-1KL149:87:HA58EADXX:1:1101:2953:2177     147
HWI-1KL149:87:HA58EADXX:1:1101:2854:2200      99
Name: FLAG, dtype: int64

In [43]:
df.FLAG

QNAME
HWI-1KL149:87:HA58EADXX:1:1101:1531:2163     163
HWI-1KL149:87:HA58EADXX:1:1101:1744:2169     141
HWI-1KL149:87:HA58EADXX:1:1101:1606:2189      83
HWI-1KL149:87:HA58EADXX:1:1101:1752:2238      83
HWI-1KL149:87:HA58EADXX:1:1101:1752:2238     163
HWI-1KL149:87:HA58EADXX:1:1101:1813:2228    2115
HWI-1KL149:87:HA58EADXX:1:1101:2031:2119      99
HWI-1KL149:87:HA58EADXX:1:1101:2133:2244     163
HWI-1KL149:87:HA58EADXX:1:1101:2228:2222     141
HWI-1KL149:87:HA58EADXX:1:1101:2344:2177      83
HWI-1KL149:87:HA58EADXX:1:1101:2645:2113     163
HWI-1KL149:87:HA58EADXX:1:1101:2485:2160     163
HWI-1KL149:87:HA58EADXX:1:1101:2647:2140      83
HWI-1KL149:87:HA58EADXX:1:1101:2813:2174      99
HWI-1KL149:87:HA58EADXX:1:1101:2953:2177     147
HWI-1KL149:87:HA58EADXX:1:1101:2854:2200      99
Name: FLAG, dtype: int64

In [44]:
df[["FLAG", "POS"]]

,FLAG,POS
QNAME,,
HWI-1KL149:87:HA58EADXX:1:1101:1531:2163,163,4862
HWI-1KL149:87:HA58EADXX:1:1101:1744:2169,141,0
HWI-1KL149:87:HA58EADXX:1:1101:1606:2189,83,4285
HWI-1KL149:87:HA58EADXX:1:1101:1752:2238,83,3032
HWI-1KL149:87:HA58EADXX:1:1101:1752:2238,163,2809
HWI-1KL149:87:HA58EADXX:1:1101:1813:2228,2115,5466
HWI-1KL149:87:HA58EADXX:1:1101:2031:2119,99,1392
HWI-1KL149:87:HA58EADXX:1:1101:2133:2244,163,4828
HWI-1KL149:87:HA58EADXX:1:1101:2228:2222,141,0


***Examples of index slicing***

In [45]:
df.loc["HWI-1KL149:87:HA58EADXX:1:1101:1531:2163"]

FLAG                                                   163
RNAME                         SSV9K2-CMV-GFP-HygroTK-bGHpA
POS                                                   4862
MAPQ                                                    60
CIGAR                                                 101M
RNEXT                                                    =
PNEXT                                                 4950
TLEN                                                   189
SEQ      CGCCCTCCTGTGCTACCCGGCCGCGCGGTACCTTATGGGCAGCATG...
QUAL     CCCFFFFFGHHHHJJJJJJJJJJJJJJJGIJJJJJJHHHHHFFFEE...
Name: HWI-1KL149:87:HA58EADXX:1:1101:1531:2163, dtype: object

In [46]:
df.loc["HWI-1KL149:87:HA58EADXX:1:1101:1531:2163":"HWI-1KL149:87:HA58EADXX:1:1101:1744:2169"]

,FLAG,RNAME,POS,MAPQ,CIGAR,RNEXT,PNEXT,TLEN,SEQ,QUAL
QNAME,,,,,,,,,,
HWI-1KL149:87:HA58EADXX:1:1101:1531:2163,163,SSV9K2-CMV-GFP-HygroTK-bGHpA,4862,60,101M,=,4950,189,CGCCCTCCTGTGCTACCCGGCCGCGCGGTACCTTATGGGCAGCATG...,CCCFFFFFGHHHHJJJJJJJJJJJJJJJGIJJJJJJHHHHHFFFEE...
HWI-1KL149:87:HA58EADXX:1:1101:1744:2169,141,*,0,0,*,*,0,0,TCGTAGATTTCTCTGGCGATTGAAGGGCTAAATTCTTCAACGCTAA...,CCCFFFFFHHHHHJJJJJJJJJJJJJJJJJJJIJJJJJJJJJJJJJ...


In [47]:
df.iloc[0]

FLAG                                                   163
RNAME                         SSV9K2-CMV-GFP-HygroTK-bGHpA
POS                                                   4862
MAPQ                                                    60
CIGAR                                                 101M
RNEXT                                                    =
PNEXT                                                 4950
TLEN                                                   189
SEQ      CGCCCTCCTGTGCTACCCGGCCGCGCGGTACCTTATGGGCAGCATG...
QUAL     CCCFFFFFGHHHHJJJJJJJJJJJJJJJGIJJJJJJHHHHHFFFEE...
Name: HWI-1KL149:87:HA58EADXX:1:1101:1531:2163, dtype: object

In [48]:
df.iloc[10:12]

,FLAG,RNAME,POS,MAPQ,CIGAR,RNEXT,PNEXT,TLEN,SEQ,QUAL
QNAME,,,,,,,,,,
HWI-1KL149:87:HA58EADXX:1:1101:2645:2113,163,SSV9K2-CMV-GFP-HygroTK-bGHpA,5493,60,101M,=,5559,167,AAATCGATGGATCCACTAGTTCTAGAGGGCCCTATTCTATAGTGTC...,CCCFFFFFHHHHHJJJJJJIIJJJJJJJJJJJJIJJJJIJJJHGII...
HWI-1KL149:87:HA58EADXX:1:1101:2485:2160,163,SSV9K2-CMV-GFP-HygroTK-bGHpA,3756,60,101M,=,3829,174,CCGGTCGCGGAGGCCATGGATGCGATCGCTGCGGCCGATCTTAGCC...,CCCFFFFFHHHHHJJJJJJJJJJJJJJJJJJJJHHFDDDDDDEDDD...


***Examples of combination of column and index slicing to select a specific item or a range of items***

In [49]:
df.loc["HWI-1KL149:87:HA58EADXX:1:1101:1531:2163":"HWI-1KL149:87:HA58EADXX:1:1101:1744:2169", "FLAG"]

QNAME
HWI-1KL149:87:HA58EADXX:1:1101:1531:2163    163
HWI-1KL149:87:HA58EADXX:1:1101:1744:2169    141
Name: FLAG, dtype: int64

In [50]:
df.loc["HWI-1KL149:87:HA58EADXX:1:1101:1531:2163"]["FLAG"]

163

In [51]:
df.iloc[0,0]

163

In [52]:
df.iloc[0][0]

163

### Select specific elements based on their values = Boolean Indexing

***With a single condition***

In [53]:
df[df.FLAG == 163]

,FLAG,RNAME,POS,MAPQ,CIGAR,RNEXT,PNEXT,TLEN,SEQ,QUAL
QNAME,,,,,,,,,,
HWI-1KL149:87:HA58EADXX:1:1101:1531:2163,163,SSV9K2-CMV-GFP-HygroTK-bGHpA,4862,60,101M,=,4950,189,CGCCCTCCTGTGCTACCCGGCCGCGCGGTACCTTATGGGCAGCATG...,CCCFFFFFGHHHHJJJJJJJJJJJJJJJGIJJJJJJHHHHHFFFEE...
HWI-1KL149:87:HA58EADXX:1:1101:1752:2238,163,SSV9K2-CMV-GFP-HygroTK-bGHpA,2809,60,101M,=,3032,324,CTATAGGGAGACCACAACGGTTTCCCTCTAGCGGGATCAATTCCGC...,CCCFFFFFHHHHHJJIJJJJIIJJJJJJJJJJJJJHJJIJJJIIIJ...
HWI-1KL149:87:HA58EADXX:1:1101:2133:2244,163,SSV9K2-CMV-GFP-HygroTK-bGHpA,4828,60,101M,=,4925,198,CCCTCACCCTAATCTTCGACCGCCATCCCATCGCCGCCCTCCTGTG...,CCCFFFFFHHHHHJJJJJJJJJJJJJJJJJJJJJJJJJJJHHHHHF...
HWI-1KL149:87:HA58EADXX:1:1101:2645:2113,163,SSV9K2-CMV-GFP-HygroTK-bGHpA,5493,60,101M,=,5559,167,AAATCGATGGATCCACTAGTTCTAGAGGGCCCTATTCTATAGTGTC...,CCCFFFFFHHHHHJJJJJJIIJJJJJJJJJJJJIJJJJIJJJHGII...
HWI-1KL149:87:HA58EADXX:1:1101:2485:2160,163,SSV9K2-CMV-GFP-HygroTK-bGHpA,3756,60,101M,=,3829,174,CCGGTCGCGGAGGCCATGGATGCGATCGCTGCGGCCGATCTTAGCC...,CCCFFFFFHHHHHJJJJJJJJJJJJJJJJJJJJHHFDDDDDDEDDD...


In [54]:
df[df.FLAG.isin([99,147])]

,FLAG,RNAME,POS,MAPQ,CIGAR,RNEXT,PNEXT,TLEN,SEQ,QUAL
QNAME,,,,,,,,,,
HWI-1KL149:87:HA58EADXX:1:1101:2031:2119,99,SSV9K2-CMV-GFP-HygroTK-bGHpA,1392,60,101M,=,1503,212,NCATTGATTATTGACTAGTTATTAATAGTAATCAATTACGGGGTCA...,#1=DDFFFHHHHHJJJJJJJJJJJJIJJIJJJJJJJJJJJJJJGHI...
HWI-1KL149:87:HA58EADXX:1:1101:2813:2174,99,SSV9K2-CMV-GFP-HygroTK-bGHpA,5279,60,101M,=,5457,279,CGTGTTTGCCTGGGCCTTGGACGTCTTGGCCAAACGCCTCCGTTCC...,CCCFFFFFHHHHGIJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJ...
HWI-1KL149:87:HA58EADXX:1:1101:2953:2177,147,SSV9K2-CMV-GFP-HygroTK-bGHpA,3530,60,101M,=,3440,-191,CTTCGATGTAGGAGGGCGTGGATATGTCCTGCGGGTAAATAGCTGC...,DDDDDDDDDDDDDDDDDDDEDEDDCDDDDDDDDDEEEDDDDDDDDD...
HWI-1KL149:87:HA58EADXX:1:1101:2854:2200,99,SSV9K2-CMV-GFP-HygroTK-bGHpA,5128,60,101M,=,5206,179,GGGAGGACTGGGGACAGCTTTCGGGGACGGCCGTGCCGCCCCAGGG...,CCCFFFFFHHHHHIJIJJJJJJJJJJJJJJJJJJJHHHFDDDDDDD...


In [55]:
df.query('FLAG == 141')

,FLAG,RNAME,POS,MAPQ,CIGAR,RNEXT,PNEXT,TLEN,SEQ,QUAL
QNAME,,,,,,,,,,
HWI-1KL149:87:HA58EADXX:1:1101:1744:2169,141,*,0,0,*,*,0,0,TCGTAGATTTCTCTGGCGATTGAAGGGCTAAATTCTTCAACGCTAA...,CCCFFFFFHHHHHJJJJJJJJJJJJJJJJJJJIJJJJJJJJJJJJJ...
HWI-1KL149:87:HA58EADXX:1:1101:2228:2222,141,*,0,0,*,*,0,0,ACAGTTTGATGAGTATAGAAATGGATCCACTCGTTATTCTCGGACG...,CCCFFFFFHHHHHHIIJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJ...


***With a multiple conditions***

In [56]:
df[(df.FLAG == 163) & (df.TLEN > 180)]

,FLAG,RNAME,POS,MAPQ,CIGAR,RNEXT,PNEXT,TLEN,SEQ,QUAL
QNAME,,,,,,,,,,
HWI-1KL149:87:HA58EADXX:1:1101:1531:2163,163,SSV9K2-CMV-GFP-HygroTK-bGHpA,4862,60,101M,=,4950,189,CGCCCTCCTGTGCTACCCGGCCGCGCGGTACCTTATGGGCAGCATG...,CCCFFFFFGHHHHJJJJJJJJJJJJJJJGIJJJJJJHHHHHFFFEE...
HWI-1KL149:87:HA58EADXX:1:1101:1752:2238,163,SSV9K2-CMV-GFP-HygroTK-bGHpA,2809,60,101M,=,3032,324,CTATAGGGAGACCACAACGGTTTCCCTCTAGCGGGATCAATTCCGC...,CCCFFFFFHHHHHJJIJJJJIIJJJJJJJJJJJJJHJJIJJJIIIJ...
HWI-1KL149:87:HA58EADXX:1:1101:2133:2244,163,SSV9K2-CMV-GFP-HygroTK-bGHpA,4828,60,101M,=,4925,198,CCCTCACCCTAATCTTCGACCGCCATCCCATCGCCGCCCTCCTGTG...,CCCFFFFFHHHHHJJJJJJJJJJJJJJJJJJJJJJJJJJJHHHHHF...


In [57]:
df.query('FLAG == 99 and TLEN > 200')

,FLAG,RNAME,POS,MAPQ,CIGAR,RNEXT,PNEXT,TLEN,SEQ,QUAL
QNAME,,,,,,,,,,
HWI-1KL149:87:HA58EADXX:1:1101:2031:2119,99,SSV9K2-CMV-GFP-HygroTK-bGHpA,1392,60,101M,=,1503,212,NCATTGATTATTGACTAGTTATTAATAGTAATCAATTACGGGGTCA...,#1=DDFFFHHHHHJJJJJJJJJJJJIJJIJJJJJJJJJJJJJJGHI...
HWI-1KL149:87:HA58EADXX:1:1101:2813:2174,99,SSV9K2-CMV-GFP-HygroTK-bGHpA,5279,60,101M,=,5457,279,CGTGTTTGCCTGGGCCTTGGACGTCTTGGCCAAACGCCTCCGTTCC...,CCCFFFFFHHHHGIJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJ...


### Iterate over a dataframe

***Iterate row by row with "iterrows"***

In [58]:
for index_name, row_values in df.iterrows():
    if row_values.CIGAR != "101M" and row_values.RNAME!="*":
        print (row_values)

FLAG                                        2115
RNAME               SSV9K2-CMV-GFP-HygroTK-bGHpA
POS                                         5466
MAPQ                                          60
CIGAR                                     39M62H
RNEXT                                          =
PNEXT                                       4511
TLEN                                        -956
SEQ      GCCCGGGAGATGGGGGAGGCTAACTGAAAATCGATGGAT
QUAL     CCCFFFFFHHHHHJJJJJJJJJJJJJJJJJJJJHJJJJJ
Name: HWI-1KL149:87:HA58EADXX:1:1101:1813:2228, dtype: object


***Iterate by group of value with "groupby"***

In [59]:
for group, group_df in df.groupby("FLAG"):
    print ("\nGroup with flag {}".format(group))
    display (group_df)


Group with flag 83


,FLAG,RNAME,POS,MAPQ,CIGAR,RNEXT,PNEXT,TLEN,SEQ,QUAL
QNAME,,,,,,,,,,
HWI-1KL149:87:HA58EADXX:1:1101:1606:2189,83,SSV9K2-CMV-GFP-HygroTK-bGHpA,4285,60,101M,=,4104,-282,AGGGTCGATGCGACGCAATCGTCCGATCCGGAGCCGGGACTGTCGG...,DDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDD...
HWI-1KL149:87:HA58EADXX:1:1101:1752:2238,83,SSV9K2-CMV-GFP-HygroTK-bGHpA,3032,60,101M,=,2809,-324,CTGTTGAATGTCGTGAAGGAAGCAGTTCCTCTGGAAGCTTCTTGAA...,DDDEEDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDEDDD...
HWI-1KL149:87:HA58EADXX:1:1101:2344:2177,83,SSV9K2-CMV-GFP-HygroTK-bGHpA,4321,60,101M,=,4198,-224,GGACTGTCGGGCGTACACAAATCGCCCGCAGAAGCGCGGCCGTCTG...,DDDDDDDDDDDDEDDDEDDDDDDDDDDDDDDDDDDDDDDDDDDDDD...
HWI-1KL149:87:HA58EADXX:1:1101:2647:2140,83,SSV9K2-CMV-GFP-HygroTK-bGHpA,4552,60,101M,=,4437,-216,CCACCACGCAACTGCTGGTGGCCCTGGGTTCGCGCGACGATATCGT...,@<2DB<BC>CABCDB?DB@B?DBA?DDDBB>@>;8?98@==?;;C=...



Group with flag 99


,FLAG,RNAME,POS,MAPQ,CIGAR,RNEXT,PNEXT,TLEN,SEQ,QUAL
QNAME,,,,,,,,,,
HWI-1KL149:87:HA58EADXX:1:1101:2031:2119,99,SSV9K2-CMV-GFP-HygroTK-bGHpA,1392,60,101M,=,1503,212,NCATTGATTATTGACTAGTTATTAATAGTAATCAATTACGGGGTCA...,#1=DDFFFHHHHHJJJJJJJJJJJJIJJIJJJJJJJJJJJJJJGHI...
HWI-1KL149:87:HA58EADXX:1:1101:2813:2174,99,SSV9K2-CMV-GFP-HygroTK-bGHpA,5279,60,101M,=,5457,279,CGTGTTTGCCTGGGCCTTGGACGTCTTGGCCAAACGCCTCCGTTCC...,CCCFFFFFHHHHGIJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJ...
HWI-1KL149:87:HA58EADXX:1:1101:2854:2200,99,SSV9K2-CMV-GFP-HygroTK-bGHpA,5128,60,101M,=,5206,179,GGGAGGACTGGGGACAGCTTTCGGGGACGGCCGTGCCGCCCCAGGG...,CCCFFFFFHHHHHIJIJJJJJJJJJJJJJJJJJJJHHHFDDDDDDD...



Group with flag 141


...

,FLAG,RNAME,POS,MAPQ,CIGAR,RNEXT,PNEXT,TLEN,SEQ,QUAL
QNAME,,,,,,,,,,
HWI-1KL149:87:HA58EADXX:1:1101:2953:2177,147,SSV9K2-CMV-GFP-HygroTK-bGHpA,3530,60,101M,=,3440,-191,CTTCGATGTAGGAGGGCGTGGATATGTCCTGCGGGTAAATAGCTGC...,DDDDDDDDDDDDDDDDDDDEDEDDCDDDDDDDDDEEEDDDDDDDDD...



Group with flag 163


,FLAG,RNAME,POS,MAPQ,CIGAR,RNEXT,PNEXT,TLEN,SEQ,QUAL
QNAME,,,,,,,,,,
HWI-1KL149:87:HA58EADXX:1:1101:1531:2163,163,SSV9K2-CMV-GFP-HygroTK-bGHpA,4862,60,101M,=,4950,189,CGCCCTCCTGTGCTACCCGGCCGCGCGGTACCTTATGGGCAGCATG...,CCCFFFFFGHHHHJJJJJJJJJJJJJJJGIJJJJJJHHHHHFFFEE...
HWI-1KL149:87:HA58EADXX:1:1101:1752:2238,163,SSV9K2-CMV-GFP-HygroTK-bGHpA,2809,60,101M,=,3032,324,CTATAGGGAGACCACAACGGTTTCCCTCTAGCGGGATCAATTCCGC...,CCCFFFFFHHHHHJJIJJJJIIJJJJJJJJJJJJJHJJIJJJIIIJ...
HWI-1KL149:87:HA58EADXX:1:1101:2133:2244,163,SSV9K2-CMV-GFP-HygroTK-bGHpA,4828,60,101M,=,4925,198,CCCTCACCCTAATCTTCGACCGCCATCCCATCGCCGCCCTCCTGTG...,CCCFFFFFHHHHHJJJJJJJJJJJJJJJJJJJJJJJJJJJHHHHHF...
HWI-1KL149:87:HA58EADXX:1:1101:2645:2113,163,SSV9K2-CMV-GFP-HygroTK-bGHpA,5493,60,101M,=,5559,167,AAATCGATGGATCCACTAGTTCTAGAGGGCCCTATTCTATAGTGTC...,CCCFFFFFHHHHHJJJJJJIIJJJJJJJJJJJJIJJJJIJJJHGII...
HWI-1KL149:87:HA58EADXX:1:1101:2485:2160,163,SSV9K2-CMV-GFP-HygroTK-bGHpA,3756,60,101M,=,3829,174,CCGGTCGCGGAGGCCATGGATGCGATCGCTGCGGCCGATCTTAGCC...,CCCFFFFFHHHHHJJJJJJJJJJJJJJJJJJJJHHFDDDDDDEDDD...



Group with flag 2115


,FLAG,RNAME,POS,MAPQ,CIGAR,RNEXT,PNEXT,TLEN,SEQ,QUAL
QNAME,,,,,,,,,,
HWI-1KL149:87:HA58EADXX:1:1101:1813:2228,2115,SSV9K2-CMV-GFP-HygroTK-bGHpA,5466,60,39M62H,=,4511,-956,GCCCGGGAGATGGGGGAGGCTAACTGAAAATCGATGGAT,CCCFFFFFHHHHHJJJJJJJJJJJJJJJJJJJJHJJJJJ


### And much more...

**Pandas documentation is extensive and updated frequently**

* [Working with Text Data](http://pandas.pydata.org/pandas-docs/stable/text.html)

* [Merge, join, and concatenate](http://pandas.pydata.org/pandas-docs/stable/merging.html)

* [Reshaping and Pivot Tables](http://pandas.pydata.org/pandas-docs/stable/reshaping.html)

* [IO Tools (Text, CSV, HDF5, ...)](http://pandas.pydata.org/pandas-docs/stable/io.html)

* [Time Series / Date functionality](pandas.pydata.org/pandas-docs/stable/timeseries.html)

* [Categorical Data](http://pandas.pydata.org/pandas-docs/stable/categorical.html)

* [Visualization plotting data](http://pandas.pydata.org/pandas-docs/stable/visualization.html)

* [Comparison with R / R libraries](http://pandas.pydata.org/pandas-docs/stable/comparison_with_r.html)